## Set up

In [1]:
%pip install --upgrade --quiet --user google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 75.7 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## ## Environment variables

In [2]:
# get project ID
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"
if PROJECT_ID == "(unset)":
    print(f"Please set the project ID manually below")

Please set the project ID manually below


In [3]:
# define project information
if PROJECT_ID == "(unset)":
    PROJECT_ID = "gen-lang-client-0974620078"  # @param {type:"string"}

# generate an unique id for this session
from datetime import datetime

UID = datetime.now().strftime("%m%d%H%M")

## Authentication (Colab only)

If you are running this notebook on Colab, you will need to run the following cell authentication. This step is not required if you are using Vertex AI Workbench as it is pre-authenticated.

In [5]:
import sys

# if it's Colab runtime, authenticate the user with Google Cloud
if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

## Create and update item to GCS

In [6]:
import pandas as pd
df = pd.read_json('/content/L21_V001.json')

In [8]:
print(len(df.embedding[0]))

768


In [26]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModel, AutoTokenizer
import requests
import numpy as np

# 1. Load model SigLIP (Vision + Text encoder) [siglip-so400m-patch14-384](https://huggingface.co/google/siglip-so400m-patch14-384)
# model_id = "google/siglip-so400m-patch14-384"
# [siglip-base-patch16-224](https://h uggingface.co/google/siglip-base-patch16-224)
model_id = "google/siglip-base-patch16-224"
tokenizer = AutoTokenizer.from_pretrained("google/siglip-base-patch16-224")
model = AutoModel.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

In [29]:
items = []
for i in range(len(df)):
    id = i
    items.append(
        {
            "id": id,
            "embedding": df['embedding'][i],
            "embedding_metadata": {
                "asr_text": df.metadata[i].get('asr_text'),
                "sound": df.metadata[i].get('sound'),
                "objects": df.metadata[i].get('objects'),
                "image_url": df.metadata[i].get('image_url'),
                "timestamp": df.metadata[i].get('timestamp'),
                "frame_idx": df.metadata[i].get('frame_idx'),
                "title": df.metadata[i].get('title'),
            }
            })
items[:5]

[{'id': 0,
  'embedding': [0.023624841123819,
   0.008479190990328001,
   0.0046246368438,
   0.059812083840370005,
   -0.036209493875503006,
   0.037103902548551004,
   -0.068388476967811,
   -0.0040090256370600005,
   -0.024195047095417,
   -0.024394100531935,
   -0.035767011344432005,
   -0.014772182330489,
   0.046630948781967003,
   0.021141106262803,
   -0.007912325672805,
   0.005585002247244001,
   0.013757143169641,
   -0.016163052991032,
   0.016968268901109,
   0.0034939167089760004,
   -0.041854761540889004,
   -0.002237109001725,
   0.005846264306455001,
   -0.003039015689864,
   -0.06409467011690101,
   -0.006960624828934001,
   -0.018551409244537003,
   0.018168324604630002,
   -0.059011876583099004,
   -0.028286859393119004,
   -0.019438523799180003,
   -0.016043471172451,
   -0.029802825301885,
   0.005912641063332001,
   0.0018585567595430002,
   0.021245131269097002,
   0.02676685899496,
   -0.06889280676841701,
   0.012106287293136,
   -0.011496733874082002,
   -0.0

In [30]:
BUCKET_URI ="gs://test-video-retrieval/item/image"
# output as a JSONL file and save to the GCS bucket
with open("items.json", "w") as f:
    for item in items:
        f.write(f"{item}\n")
! gsutil cp items.json $BUCKET_URI


Copying file://items.json [Content-Type=application/json]...
|
Operation completed over 1 objects/5.0 MiB.                                      


## Build and Deploy a Vector Search Index

In [31]:
# init the aiplatform package
from google.cloud import aiplatform

LOCATION = "asia-southeast1"
aiplatform.init(project="gen-lang-client-0974620078", location=LOCATION)

In [144]:

INDEX_NAME = f"ai-challenge-image-{UID}"
# create Index
# my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
#     display_name=INDEX_NAME,    # index name
#     dimensions=768,
#     contents_delta_uri='gs://test-video-retrieval/item/image',
#     approximate_neighbors_count=20, # how many similar items we want to retrieve in typical cases
# )

# Get an existing index
my_index_id = "8043244216530239488"  # @param {type:"string"}
my_index = aiplatform.MatchingEngineIndex(my_index_id)

In [33]:
# Get an existing Index Endpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint("5237105824492421120")

In [34]:
DEPLOYED_INDEX_ID = f"ai_challenge_image_deployed_{UID}"
# deploy the Index to the Index Endpoint
# my_index_endpoint.deploy_index(index=my_index, deployed_index_id=DEPLOYED_INDEX_ID)

resource name: projects/284454080854/locations/asia-southeast1/indexEndpoints/5237105824492421120

In [159]:
query_text = "four women"
def embedding_text(texts):
    with torch.no_grad():
        inputs = processor(text=texts, return_tensors="pt", padding=True)
        embeds = model.get_text_features(**inputs)
        embeds = embeds / embeds.norm(dim=-1, keepdim=True)
    return embeds.cpu().numpy().tolist()

embed_text = embedding_text(query_text)

In [160]:
# Test query
response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=embed_text,
    num_neighbors=20,
)

In [162]:
import numpy as np

for idx, neighbor in enumerate(response[0]):
    id = np.int64(neighbor.id)
    # Use iloc to access the row by index
    similar = df.iloc[id]
    print(similar)
    print(similar.metadata)
    print(f"{neighbor.distance:.4f}")

id                                               L21_V001_F299
embedding    [0.024627586826682, -0.013225213624536, 0.0244...
metadata     {'id': 'L21_V001_F299', 'timestamp': '00:20:25...
Name: 298, dtype: object
{'id': 'L21_V001_F299', 'timestamp': '00:20:25', 'frame_idx': 36764, 'image_url': 'https://storage.googleapis.com/test-video-retrieval/dataset/unzips/Keyframes_L21/keyframes/L21_V001/299.jpg', 'asr_text': 'theo thông tin ban đầu khoảng mười tám giờ bốn mươi lăm phúc xe máy do một người đàn ông cầm lái đang di chuyển từ đường tô ký húng về đường nguyễn thị búc khi phương tiện này đến trước địa chỉ số một ngàn không trăm bốn mươi lăm đường nguyễn ảnh thụng phường tân chánh hiệp thì bất ngờ xảy ra va chạm với xe tải chạy cùng chiều chiếc xe tải cán qua người khiến nạn nhân tử vong tại chỗ', 'sound': 'Speech:0.97;Conversation:0.02;Music:0.01', 'objects': {'Man': 2, 'Clothing': 1, 'Person': 3, 'Land vehicle': 2, 'Woman': 1, 'Building': 1}, 'title': '60 Giây Sáng - Ngày 01082024 - 